## Day 2 Exercise
building upon the day1 exercise to offer a multi models via dropdown.
externalized the common methods into a AISystem.py file to be reused down the line

## Load env, check for api keys and load up the connections

In [ ]:
# !pip install -r requirements.txt
# !pip freeze > requirements.txt

In [ ]:
import os
from enum import Enum, auto
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from AISystem import formatPrompt, AI, AISystem
import gradio as gr # oh yeah!

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

openai = OpenAI()

claude = anthropic.Anthropic()

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
openai_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"
ollama_model = "deepseek-r1:8b"

In [ ]:
with open("my_tools.txt", "r", encoding="utf-8") as f:
    tools = f.read()

prompt = f"""
You are an AI agent specializing in concept development and deconstruction of
instructions into detailed, step-by-step tasks. \n
You have the following tools at your disposal: {tools}. \n
If the concept involves the use of tools beyond those listed, suggest using other tools.
"""

## Create the systems for each llms

In [ ]:
geminiSys = AISystem(gemini_via_openai_client,
                        formatPrompt("system",prompt), 
                        gemini_model,
                        AI.GEMINI)

openAiSys = AISystem(openai,
                        formatPrompt("system",prompt), 
                        openai_model,
                        AI.OPEN_AI)

claudeSys = AISystem(claude,
                       prompt, 
                       claude_model,
                        AI.CLAUDE)

ollamaSys = AISystem(ollama_via_openai,
                        formatPrompt("system",prompt), 
                        ollama_model,
                        AI.OLLAMA)
sys_dict = { AI.GEMINI: geminiSys, AI.OPEN_AI: openAiSys, AI.CLAUDE: claudeSys, AI.OLLAMA: ollamaSys}

def stream_model(prompt, model):
    aiSystem = sys_dict.get(AI[model.upper()])
    yield from aiSystem.stream(formatPrompt("user",prompt), True)

## Create the gradio interface linking with the AI enum for the dropdown

In [ ]:

view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your prompt:", lines=12) , gr.Dropdown(choices=[ai.value for ai in AI], label="Select model")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()